In [1]:
!export SPARK_LOCAL_DIRS="/data/tmp/conner"
!export PYSPARK_PYTHON="/home/connermanuel/.cache/pypoetry/virtualenvs/cider-pYyYiFm9-py3.8/bin/python"
!export PYSPARK_DRIVER_PYTHON="/home/connermanuel/.cache/pypoetry/virtualenvs/cider-pYyYiFm9-py3.8/bin/python"

In [2]:
from cider.featurizer import Featurizer
from cider.datastore import DataStore
import pandas as pd
import pyspark

In [3]:
cfg_file = './config.yml'
ds = DataStore(cfg_file)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/05 13:00:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
featurizer = Featurizer(ds)

Loading CDR...


['interaction', 'caller_msisdn', 'recipient_msisdn', 'datetime', 'duration', 'antenna_id', 'recipient_type']
['txn_type', 'caller_id', 'recipient_id', 'timestamp', 'duration', 'caller_antenna', 'international']


['antenna_id', 'latitude', 'longitude', 'canton', 'prefecture', 'region', 'mno', 'imputed']
['antenna_id', 'latitude', 'longitude']
+----------+--------+---------+
|antenna_id|latitude|longitude|
+----------+--------+---------+
|     11266|6.143371| 1.259347|
|     11265|6.143371| 1.259347|
|     11264|6.143371| 1.259347|
|      5011|6.143371| 1.259347|
|      5012|6.143371| 1.259347|
|      5013|6.143371| 1.259347|
|      5014|6.143371| 1.259347|
|      5015|6.143371| 1.259347|
|      5016|6.143371| 1.259347|
|      5019|6.143371| 1.259347|
|      5018|6.143371| 1.259347|
|      5017|6.143371| 1.259347|
|     13563|6.136645| 1.257274|
|     13562|6.136645| 1.257274|
|     13561|6.136645| 1.257274|
|      5131|6.136645| 1.257274|
|      5132|6.136645| 1.257274|
|      5133|6.136645| 1.257274|
|      5134|6.136645| 1.257274|
|      5135|6.136645| 1.257274|
+----------+--------+---------+
only showing top 20 rows

Columns: Index(['OBJECTID', 'region', 'NBRE_CANT', 'geometry'], dtype='obj

In [5]:
# add null column of recipient_antenna
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType
featurizer.ds.cdr = featurizer.ds.cdr.withColumn('recipient_antenna', lit(None).cast(StringType()))
featurizer.cfg.col_names.cdr['recipient_antenna'] = 'recipient_antenna'

# Try with first 1000 rows

In [6]:
# # Try with first 1000 rows
# full_df = featurizer.ds.cdr
# featurizer.ds.cdr = full_df.limit(100)

In [7]:
# featurizer.ds.cdr.count()

In [8]:
# cdr_features = featurizer.cdr_features_spark()

In [9]:
# featurizer.ds.antennas.show(5)

In [10]:
# featurizer.ds.cdr

In [11]:
# # TODO:
# featurizer.location_features()
# featurizer.international_features()

In [12]:
# featurizer.all_features()

# Full Featurization

In [16]:
# featurizer.ds.cdr = full_df
featurizer.ds.cdr.count()

3546896

In [17]:
featurizer.cdr_features_spark()

Calculating CDR features...


In [18]:
featurizer.location_features()

Calculating spatial features...
['interaction', 'name', 'correspondent_id', 'datetime', 'call_duration', 'antenna_id', 'international', 'direction']
['interaction', 'correspondent_id', 'name', 'datetime', 'call_duration', 'international', 'antenna_id', 'antenna_id', 'direction']
['interaction', 'correspondent_id', 'name', 'datetime', 'call_duration', 'international', 'antenna_id', 'antenna_id', 'direction']


/home/connermanuel/.cache/pypoetry/virtualenvs/cider-pYyYiFm9-py3.8/lib64/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/connermanuel/cider/cider/featurizer.py:332: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  antennas = gpd.sjoin(antennas, shapefile, op='within', how='left').drop('index_right', axis=1)
/home/connermanuel/cider/cider/featurizer.py:332: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproj

<class 'geopandas.geodataframe.GeoDataFrame'>
Index(['antenna_id', 'canton', 'prefecture', 'region', 'mno', 'imputed',
       'regions', 'cantons', 'prefectures'],
      dtype='object')
['antenna_id', 'canton', 'prefecture', 'region', 'mno', 'imputed', 'regions', 'cantons', 'prefectures']


/home/connermanuel/cider/cider/featurizer.py:367: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  count_by_region[c + '_percent'] = count_by_region[c] / count_by_region['total']
/home/connermanuel/cider/cider/featurizer.py:367: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  count_by_region[c + '_percent'] = count_by_region[c] / count_by_region['total']
/home/connermanuel/cider/cider/featurizer.py:367: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor

/home/connermanuel/cider/cider/featurizer.py:367: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  count_by_region[c + '_percent'] = count_by_region[c] / count_by_region['total']
/home/connermanuel/cider/cider/featurizer.py:367: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  count_by_region[c + '_percent'] = count_by_region[c] / count_by_region['total']
/home/connermanuel/cider/cider/featurizer.py:367: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor

...

/home/connermanuel/cider/cider/featurizer.py:367: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  count_by_region[c + '_percent'] = count_by_region[c] / count_by_region['total']
/home/connermanuel/cider/cider/featurizer.py:367: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  count_by_region[c + '_percent'] = count_by_region[c] / count_by_region['total']
/home/connermanuel/cider/cider/featurizer.py:367: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor

/home/connermanuel/cider/cider/featurizer.py:367: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  count_by_region[c + '_percent'] = count_by_region[c] / count_by_region['total']
/home/connermanuel/cider/cider/featurizer.py:367: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  count_by_region[c + '_percent'] = count_by_region[c] / count_by_region['total']
/home/connermanuel/cider/cider/featurizer.py:367: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor

/home/connermanuel/cider/cider/featurizer.py:367: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  count_by_region[c + '_percent'] = count_by_region[c] / count_by_region['total']
/home/connermanuel/cider/cider/featurizer.py:367: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  count_by_region[c + '_percent'] = count_by_region[c] / count_by_region['total']
/home/connermanuel/cider/cider/featurizer.py:367: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor

/home/connermanuel/cider/cider/featurizer.py:367: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  count_by_region[c + '_percent'] = count_by_region[c] / count_by_region['total']
/home/connermanuel/cider/cider/featurizer.py:367: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  count_by_region[c + '_percent'] = count_by_region[c] / count_by_region['total']
/home/connermanuel/cider/cider/featurizer.py:367: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor

/home/connermanuel/cider/cider/featurizer.py:367: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  count_by_region[c + '_percent'] = count_by_region[c] / count_by_region['total']
/home/connermanuel/cider/cider/featurizer.py:367: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  count_by_region[c + '_percent'] = count_by_region[c] / count_by_region['total']
/home/connermanuel/cider/cider/featurizer.py:367: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor

In [19]:
featurizer.international_features()

Calculating international features...


In [34]:
del featurizer.features['cdr_list']

In [35]:
featurizer.all_features()

In [ ]:
featurizer.features['all'].select(featurizer.features['all'].columns[:4]).show()

In [ ]:
featurizer.diagnostic_plots()

In [ ]:
featurizer.feature_plots()

In [20]:
# featurizer.all_features()

# Adding Labels

In [36]:
label_fpath = '/data/togo_anon/surveys/endline/fsec.csv'

In [37]:
label_df = featurizer.spark.read.csv(label_fpath, inferSchema=True, header=True)

In [38]:
label_df.show(5)

+----------------+------+----------+------+-------+--------+------------------+----+--------------------+---------+---+
|    phone_number|1meals|2preferred|3limit|4reduce|5reducec|           6mealsc|7buy|          fsec_index|treatment|uid|
+----------------+------+----------+------+-------+--------+------------------+----+--------------------+---------+---+
|o7zAGNlD6X4rVQKa|   0.0|      0.25|  0.25|    0.0|     0.0|0.3333333333333333| 0.0| -0.7257207768326157|        1|  0|
|pKy2r1NKw99mv0oD|   0.0|      0.25|  0.25|    0.0|     0.0|0.3333333333333333| 0.0| -0.7257207768326157|        0|  1|
|YRV0m8W0gonr3E4g|   0.0|       1.0|  0.25|   0.25|    null|              null| 0.0|-0.42853823493545007|        1|  2|
|5Kowry7neRKGYReA|   0.0|      0.75|  0.75|   0.75|     1.0|0.3333333333333333| 1.0|   0.639351452963377|        0|  3|
|6wd0G5YAVQWGj3ZV|   0.0|      0.25|  0.25|    0.0|     0.0|               0.0| 0.0| -0.7257207768326157|        1|  4|
+----------------+------+----------+----

In [39]:
featurizer.features['all'].select(featurizer.features['all'].columns[:4]).show(5)

+----------------+------------------------------+---------------------------+-----------------------------+
|            name|cdr_active_days_allweek_allday|cdr_active_days_allweek_day|cdr_active_days_allweek_night|
+----------------+------------------------------+---------------------------+-----------------------------+
|0m8YwMeYpl5vj9wG|                             1|                       null|                            1|
|1DvGgJVKKbXOmyw0|                             2|                          2|                            1|
|1DvGgJVxnDjYmyw0|                             2|                          1|                            1|
|3NvAj0qJWr4PxzQk|                             7|                          3|                            4|
|3Od9wzGlNrxe5Rny|                            12|                          9|                            3|
+----------------+------------------------------+---------------------------+-----------------------------+
only showing top 5 rows



In [40]:
joined = featurizer.features['all'].join(label_df, featurizer.features['all'].name == label_df.phone_number)

In [41]:
joined.select(joined.columns[:4]).show()

+----------------+------------------------------+---------------------------+-----------------------------+
|            name|cdr_active_days_allweek_allday|cdr_active_days_allweek_day|cdr_active_days_allweek_night|
+----------------+------------------------------+---------------------------+-----------------------------+
|5KowryN1d6WmYReA|                           124|                        100|                           64|
|5KowryNL7P1mYReA|                            74|                         62|                           35|
|5KowryNwyj8mYReA|                           165|                        155|                          123|
|5KowryNxYR5mYReA|                            69|                         64|                           32|
|5KowryOxL3EGYReA|                            12|                          4|                           10|
|6wd0G5YKR0lGj3ZV|                           144|                        119|                          107|
|6wd0G5YQ926Gj3ZV|          

In [42]:
joined.count()

11392

In [43]:
from helpers.utils import save_df
joined_fpath = '/data/togo_anon/tmp/cdr_impact_eval/filtered_cdr.csv'
save_df(joined, joined_fpath)

## TODO:
* count how many matching phone numbers there are from fsec data and featurized model
* use cider learner package to train supervised models, predicting fsec data
* untuned rf -> tuned rf
* display metrics